In [1]:
import pandas as pd
import numpy as np
from plotly import tools
import plotly.graph_objs as go

In [2]:
def dataset_per_country_code(df, prov):
    df_prov = df[df['PROVINCIA']==prov]
    df_prov.drop('PROVINCIA', axis=1, inplace=True)
    df_prov.set_index('DATA',inplace=True)
    df_prov=df_prov.asfreq('3H').reset_index()
    df_prov.set_index('DATA',inplace=True)
    dfprov = df[df['PROVINCIA']==prov]
    dfprov['DATA']=pd.DatetimeIndex(dfprov['DATA'])
    dfprov.drop('PROVINCIA', axis=1, inplace=True)
    dfprov.set_index('DATA',inplace=True)
    df_final=pd.merge(dfprov, df_prov['TIPO_GIORNO_DS'], left_index=True, right_index=True, how='outer')
    df_final.drop('TIPO_GIORNO_DS_y', axis=1, inplace=True)
    df_final.rename(columns={'TIPO_GIORNO_DS_x' : 'TIPO_GIORNO_DS'}, inplace=True)
    df_final[['MAX_DS','SOPRA_SOGLIA_GIALLA','SOPRA_SOGLIA_ROSSA', 'SOPRA_CAPACITA', 'N_APPARATI','KPI']]=df_final[['MAX_DS','SOPRA_SOGLIA_GIALLA','SOPRA_SOGLIA_ROSSA', 'SOPRA_CAPACITA', 'N_APPARATI','KPI']].fillna(0)
    df_final[['TIPO_GIORNO_DS']]=df_final[['TIPO_GIORNO_DS']].fillna(method='ffill')
    df_final['PRED'] = df_final['MAX_DS'].shift(-1)
    df_final.PRED.fillna(0, inplace=True)
    return df_final

In [10]:
df = pd.read_csv('./datasets/to_train.csv', keep_default_na=False, na_values=['_'])
df.drop('REGIONE',axis=1,inplace=True)
df_f=dataset_per_country_code(df,'RM')

C:\Users\ant.pagano\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
df_na_week1=df_f[(df_f.index>='2019-12-23 00:00') & (df_f.index<='2019-12-29 23:45')]

df_na_week2=df_f[(df_f.index>='2019-12-30 00:00') & (df_f.index<='2020-01-05 23:45')]

df_na_week3=df_f[(df_f.index>='2020-01-06 00:00') & (df_f.index<='2020-01-12 23:45')]

df_na_week4=df_f[(df_f.index>='2020-01-13 00:00') & (df_f.index<='2020-01-19 23:45')]

df_na_week5=df_f[(df_f.index>='2020-01-20 00:00') & (df_f.index<='2020-01-26 23:45')]

m=max(df_na_week1.MAX_DS.max(),df_na_week2.MAX_DS.max(),df_na_week3.MAX_DS.max(),df_na_week4.MAX_DS.max(),
    df_na_week1.MAX_DS.max())

In [12]:
#time series by 3 hours
trace2 = go.Scatter(x = df_na_week2.index ,y = df_na_week2.MAX_DS, name = "SUM(MAX_DS)", line = dict(color = '#7F7F7F'), opacity = 0.8)
trace3 = go.Scatter(x = df_na_week3.index ,y = df_na_week3.MAX_DS, name = "SUM(MAX_DS)", line = dict(color ='#32CD32'), opacity = 0.8)
trace4 = go.Scatter(x = df_na_week4.index ,y = df_na_week4.MAX_DS, name = "SUM(MAX_DS)", line = dict(color ='#AD9B0C'), opacity = 0.8)
trace5 = go.Scatter(x = df_na_week5.index ,y = df_na_week5.MAX_DS, name = "SUM(MAX_DS)", line = dict(color ='#B22222'), opacity = 0.8)

fig = tools.make_subplots(rows=4, cols=1, specs=[[{}], [{}], [{}], [{}]])

fig.append_trace(trace2, 1,1)
fig.append_trace(trace3, 2,1)
fig.append_trace(trace4, 3,1)
fig.append_trace(trace5, 4,1)

fig['layout'].update( height=900,title='Time Series Max_DS provincia di NA', showlegend=False)
fig['layout']['yaxis1'].update(title='', range=[0, m], autorange=False)
fig['layout']['yaxis2'].update(title='', range=[0, m], autorange=False)
fig['layout']['yaxis3'].update(title='', range=[0, m], autorange=False)
fig['layout']['yaxis4'].update(title='', range=[0, m], autorange=False)

fig.show()

In [13]:
df_na_week3.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 56 entries, 2020-01-06 00:00:00 to 2020-01-12 21:00:00
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TIPO_GIORNO_DS       56 non-null     float64
 1   MAX_DS               56 non-null     float64
 2   SOPRA_SOGLIA_GIALLA  56 non-null     float64
 3   SOPRA_SOGLIA_ROSSA   56 non-null     float64
 4   SOPRA_CAPACITA       56 non-null     float64
 5   N_APPARATI           56 non-null     float64
 6   KPI                  56 non-null     float64
 7   PRED                 56 non-null     float64
dtypes: float64(8)
memory usage: 3.9 KB


In [14]:
df_na_week2.to_csv (r'./datasets/tm_na1.csv', index = True, header=True, float_format='%.3f')
df_na_week3.to_csv (r'./datasets/tm_na2.csv', index = True, header=True, float_format='%.3f')
df_na_week4.to_csv (r'./datasets/tm_na3.csv', index = True, header=True, float_format='%.3f')
df_na_week5.to_csv (r'./datasets/tm_na4.csv', index = True, header=True, float_format='%.3f')